<a href="https://colab.research.google.com/github/mille055/AIPI590-XAI/blob/main/Assignments/08_explainable_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href='https://ai.meng.duke.edu'> = <img align="left" style="padding-top:10px;" src=https://storage.googleapis.com/aipi_datasets/Duke-AIPI-Logo.png>

# AIPI 590 - XAI | Assignment 08

#Description: Interpretable LLM
This notebook is for exploring explainability of LLMs. Specifically, I am looking at perturbation methods for chatgpt 3 model on the eighth grade math dataset. I had originally beeen trying to analyze a dataset I created and an LLM model I finetuned for the purposes of deciding which type of CT to perform for a given set of clinical paramaters and trying to do counterfactual examples, but this was not working so instead am looking at this more straightforward problem (the math Q and A).


## Chad Miller

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mille055/AIPI590-XAI/blob/main/Assignments/08_explainable_llm.ipynb)

Acknowledgements: Class Repository code on GAM model, kaggle telco-customer-churn dataset

In [43]:
import os

# Remove Colab default sample_data
!rm -r /content/sample_data

# Clone GitHub files to colab workspace
repo_name = "AIPI590-XAI"
git_path = 'https://github.com/mille055/AIPI590-XAI.git'
!git clone "{git_path}"
# !git clone 'https://github.com/mille055/CT_Protocol.git'

# Install dependencies from requirements.txt file
!pip install -r "{os.path.join(repo_name,'requirements.txt')}"
# !pip install -r "{os.path.join(repo_name, 'requirements2.txt')}"

notebook_dir = 'Assignments'
path_to_notebook = os.path.join(repo_name,notebook_dir)


rm: cannot remove '/content/sample_data': No such file or directory
fatal: destination path 'AIPI590-XAI' already exists and is not an empty directory.
  Cloning https://github.com/christophM/rulefit.git to /tmp/pip-install-ad73onf5/rulefit_a5f61073db924d2eae104e9cf71159ae
  Running command git clone --filter=blob:none --quiet https://github.com/christophM/rulefit.git /tmp/pip-install-ad73onf5/rulefit_a5f61073db924d2eae104e9cf71159ae
  Resolved https://github.com/christophM/rulefit.git to commit 472b8574b4eb9e565caf1e05ed580998fe2c9a8e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/MaximeJumelle/ALEPython.git to /tmp/pip-install-ad73onf5/alepython_69bb84ce5aec41a08b5134388ca71d92
  Running command git clone --filter=blob:none --quiet https://github.com/MaximeJumelle/ALEPython.git /tmp/pip-install-ad73onf5/alepython_69bb84ce5aec41a08b5134388ca71d92
  Resolved https://github.com/MaximeJumelle/ALEPython.git to commit 286350ab674980a32270db2a0b5ccca1380312a7
  Prepar

In [66]:
## Standard libraries
import json
import math
import time
import numpy as np
import tabulate
import urllib.request
import zipfile
import pandas as pd
import re


## Imports for plotting
from IPython.display import Image
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
#set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.set()


## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

# Torchvision
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms, datasets
import torchvision.transforms.functional as TF
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
from torchvision.datasets import ImageNet

# Other
from huggingface_hub import login
from datasets import load_dataset
from sklearn.metrics import confusion_matrix, classification_report
from openai import OpenAI
from getpass import getpass
from google.colab import userdata

In [67]:
api_key = None

# Try to load the API key from an environment variable
api_key = userdata.get('OPENAI_API_KEY')

# If the API key is not found,
if not api_key:
    print('not in environment variables...')
    api_key = getpass("Enter your OpenAI API key: ")

if api_key:
  print('API key found')



client = OpenAI(api_key=api_key)



API key found


In [71]:

# Load the eigth grade math dataset from Hugging Face Hub
dataset = load_dataset("gsm8k", 'main', split="train")

# making dataframe version
df = dataset.to_pandas()
# Dataset size
print(f'Downloaded dataset length is ', len(dataset))


# limiting to first 200
print('Limiting to the first 200 Q A pairs for this analysis.')
print('Some example question and answers:')
df = df[:200]
df.head()


Downloaded dataset length is  7473
Limiting to the first 200 Q A pairs for this analysis.
Some example question and answers:


,question,answer
0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...
1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...
2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<..."
3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....
4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...


In [72]:
# Function to parse the correct answer from the structured answer text
def parse_correct_answer(entry):
    answer_text = entry['answer']

    # Using regex to find the final answer after '####'
    match = re.search(r'####\s*(\d+)', answer_text)
    if match:
        # Converting answer to integer
        return int(match.group(1))
    return None



In [85]:
# Call the OpenAI API to get a response
def get_gpt3_response(prompt):

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. For the problem, think step by step. After solving the problem, but the answer in the format numerical answer /n####"},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0.3
    )
    # Extracting and returning the model's response
    return response.choices[0].message.content.strip()



In [82]:
# Example of function use
question = df.iloc[0]['question']
response = get_gpt3_response(question)
print("Response:", response)

Response: In April, Natalia sold 48 clips.
In May, she sold half as many clips as in April, which is 48 / 2 = 24 clips.
Altogether, Natalia sold 48 + 24 = 72 clips in April and May.


In [77]:
# Function to assess answers and get model responses
def assess_math_questions(df):
    results = []
    for index, row in df.iterrows():
        question = row['question']
        correct_answer = parse_correct_answer(row)
        model_response = get_gpt3_response(question)
        model_numerical_answer = int(re.findall(r'\d+', model_response.strip())[-1]) if re.findall(r'\d+', model_response.strip()) else None

        # Storing results in a dictionary for easier analysis later
        results.append({
            'question': question,
            'correct_answer': correct_answer,
            'model_response': model_response,
            'model_numerical_answer': model_numerical_answer,
            'is_correct': correct_answer == model_numerical_answer,
        })

    return pd.DataFrame(results)



In [86]:
# Run assessment and view results
results_df = assess_math_questions(df)
results_df.head()
print(results_df.head())

# saving results as csv
results_df.to_csv('math_results.csv', index=False)

                                            question  correct_answer  \
0  Natalia sold clips to 48 of her friends in Apr...              72   
1  Weng earns $12 an hour for babysitting. Yester...              10   
2  Betty is saving money for a new wallet which c...               5   
3  Julie is reading a 120-page book. Yesterday, s...              42   
4  James writes a 3-page letter to 2 different fr...             624   

                                      model_response  model_numerical_answer  \
0  In April, Natalia sold 48 clips.\nIn May, she ...                      72   
1  First, we need to convert 50 minutes to hours....                      10   
2  1. Betty needs $100 for the wallet.\n2. She ha...                       5   
3  Julie read 12 pages yesterday and twice as man...                      42   
4  To find out how many pages James writes in a y...                     624   

   is_correct  
0        True  
1        True  
2        True  
3        True  
4     

In [87]:
print('Accuracy of the model:', results_df['is_correct'].mean())


Accuracy of the model: 0.59


In [93]:
import random

In [129]:
# function to randomize red herring
def generate_red_herring():
    # Defining red herring templates
    herring_statements = [
        "Roger likes apples.",
        "In another room, there were more unrelated to this.",
        "Halloween is her favorite holiday.",
        "She also likes candy.",
        "He wanted to dress up as a vampire."
    ]
    return random.choice(herring_statements)


# Function to insert the red herring before the final question
def insert_herring_before_final_question(question):
    # Splitting the text into sentences
    sentences = re.split(r'(?<=[.!?]) +', question)

    # Check if the last sentence is a question
    if len(sentences) > 1 and sentences[-1].endswith("?"):
        herring = generate_red_herring()
        # Insert the herring just before the final question
        sentences.insert(-1, herring)

    # Rejoin sentences into a single string
    return ' '.join(sentences)


def rearrange_statements(question):
    # Split the text into sentences
    sentences = re.split(r'(?<=[.!?]) +', question)
    sentences_to_swap = sentences[:-1]
    random.shuffle(sentences_to_swap)
    shuffled = " ".join(sentences_to_swap)
    final = shuffled + ' ' + sentences[-1]
    #print(final)
    return final


In [130]:
sample_question = df.iloc[5]['question']
print('Sample question:', sample_question)
print('Perturbed question:', insert_herring_before_final_question(sample_question))
print('Rearranged question:', rearrange_statements(sample_question))

Sample question: Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?
Perturbed question: Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. Roger likes apples. How many flowers does Mark have in his garden?
Rearranged question: There are only 25% as many green flowers as there are yellow and purple flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. Mark has a garden with flowers. How many flowers does Mark have in his garden?


In [159]:
altered_dataframe = df.copy()
altered_dataframe['question'] = altered_dataframe['question'].apply(insert_herring_before_final_question)
#altered_dataframe['question'] = altered_dataframe['question'].apply(rearrange_statements)
altered_dataframe.head()

,question,answer
0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...
1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...
2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<..."
3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....
4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...


In [160]:
# Run assessment and view results
altered_results_df = assess_math_questions(altered_dataframe)
altered_results_df.head()
print(altered_results_df.head())

# saving results as csv
altered_results_df.to_csv('math_results_altered.csv', index=False)

                                            question  correct_answer  \
0  Natalia sold clips to 48 of her friends in Apr...              72   
1  Weng earns $12 an hour for babysitting. Yester...              10   
2  Betty is saving money for a new wallet which c...               5   
3  Julie is reading a 120-page book. Yesterday, s...              42   
4  James writes a 3-page letter to 2 different fr...             624   

                                      model_response  model_numerical_answer  \
0  In April, Natalia sold 48 clips.\nIn May, she ...                      72   
1  First, we need to convert 50 minutes into hour...                      10   
2  First, let's calculate how much money Betty ha...                       5   
3  First, let's calculate how many pages Julie re...                      42   
4  First, let's find out how many letters James w...                     624   

   is_correct  
0        True  
1        True  
2        True  
3        True  
4     

In [161]:
print('Accuracy of the model:', altered_results_df['is_correct'].mean())


Accuracy of the model: 0.6


In [162]:
response_baseline = results_df['model_response']

response_altered = altered_results_df['model_response']

predicted_answers_baseline = results_df['model_numerical_answer']
predicted_answers_altered = altered_results_df['model_numerical_answer']

correct_answers = results_df['correct_answer']

# make dataframe from these values:
comparison_df = pd.DataFrame({
    'question': df['question'],
    'correct_answer': correct_answers,
    'model_response_baseline': response_baseline,
    'model_response_altered': response_altered,
    'predicted_answer_baseline': predicted_answers_baseline,
    'predicted_answer_altered': predicted_answers_altered,
})

comparison_df.head()

,question,correct_answer,model_response_baseline,model_response_altered,predicted_answer_baseline,predicted_answer_altered
0,Natalia sold clips to 48 of her friends in Apr...,72,"In April, Natalia sold 48 clips.\nIn May, she ...","In April, Natalia sold 48 clips.\nIn May, she ...",72,72
1,Weng earns $12 an hour for babysitting. Yester...,10,"First, we need to convert 50 minutes to hours....","First, we need to convert 50 minutes into hour...",10,10
2,Betty is saving money for a new wallet which c...,5,1. Betty needs $100 for the wallet.\n2. She ha...,"First, let's calculate how much money Betty ha...",5,5
3,"Julie is reading a 120-page book. Yesterday, s...",42,Julie read 12 pages yesterday and twice as man...,"First, let's calculate how many pages Julie re...",42,42
4,James writes a 3-page letter to 2 different fr...,624,To find out how many pages James writes in a y...,"First, let's find out how many letters James w...",624,624


In [163]:
discrepant_responses = comparison_df[comparison_df['predicted_answer_baseline'] != comparison_df['predicted_answer_altered']]

#print(discrepant_responses[['question', 'correct_answer', 'model_response_baseline', 'predicted_answer_baseline']])

In [164]:

print("Example responses where the answers were discrepant:")

sample = discrepant_responses[:10]
for index, row in sample.iterrows():
  print(f'Question: {index}', row['question'])
  print('Correct answer:', row['correct_answer'])
  print('Baseline response:', row['model_response_baseline'])
  print('Perturbed response:', row['model_response_altered'])
  print('Baseline prediction:', row['predicted_answer_baseline'])
  print('Perturbed prediction:', row['predicted_answer_altered'])
  print('\n')

Example responses where the answers were discrepant:
Question: 5 Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?
Correct answer: 35
Baseline response: 1. First, we find out how many purple flowers Mark has. Since there are 80% more purple flowers than yellow flowers, we calculate 80% of 10 (yellow flowers) which is 0.80 * 10 = 8. Adding this to the number of yellow flowers gives us 10 + 8 = 18 purple flowers.

2. Next, we need to find out how many green flowers Mark has. Since there are 25% as many green flowers as yellow and purple flowers combined, we calculate 25% of the total number of yellow and purple flowers, which is 0.25 * (10 + 18) = 0.25 * 28 = 7 green flowers.

3.
Perturbed response: Let's start by finding out how many purple flowers Mark has in hi

# Analyze results
